In [0]:
# Install Java, Spark, and Findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [18]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2020-05-18 01:08:13--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.3’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.05MB/s    in 0.8s    

2020-05-18 01:08:14 (1.05 MB/s) - ‘postgresql-42.2.9.jar.3’ saved [914037/914037]



In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [23]:
#load video game reviews data from s3.amazonaws.com
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"

spark.sparkContext.addFile(url)

video_games_df = spark.read.option("header","true").csv(SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv.gz"), inferSchema=True, sep="\t")
video_games_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...|2015-08-31 00:00:00|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...| 

## Drop duplicates and incomplete rows

In [24]:
#drop duplicates and incomplete rows
print(video_games_df.count())
video_games_df = video_games_df.dropna()
print(video_games_df.count())
video_games_df = video_games_df.dropDuplicates()
#show total counts of reviews after dropping duplicates and incomplete rows
print(video_games_df.count())

1785997
1785886
1785886


## Examine the schema

In [25]:
video_games_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)



## Create a new DataFrame for vine reviews info

In [26]:
vine_reviews = video_games_df.select(["review_id","star_rating","helpful_votes","total_votes","vine"])
vine_reviews.show(10)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R100E0YCS06YNR|          5|            0|          0|   N|
|R100HR44BSB9I4|          1|            1|          1|   N|
|R100UABZOUF1JP|          1|            0|          0|   N|
|R101MOW8A5WC4J|          5|            0|          0|   N|
|R101X7KSBGIWF1|          1|            0|          0|   N|
|R101ZTSTDC8KQ4|          5|            0|          0|   N|
|R103Q4U55JZAD1|          5|            0|          0|   N|
|R103SW6BLR1HH6|          5|            0|          0|   N|
|R1048AZPZ1XQVY|          4|            0|          0|   N|
|R104C56FZRE21Q|          5|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 10 rows



In [33]:
#select vine reviews
vine_reviews_yes = vine_reviews.filter(vine_reviews["vine"]=="Y")
#show the total count of vine reviews
print(vine_reviews_yes.count())
vine_reviews_yes.show(10)

4290
+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R2XVQGWM2ZBPVC|          2|            0|          1|   Y|
| RY4HCU6JP68XV|          4|            3|          4|   Y|
| RP4G580BS1BUW|          3|            3|          5|   Y|
| R7UVLCPUNNEH5|          5|            0|          0|   Y|
|R2KC8PGKFPTDZE|          5|            0|          0|   Y|
| R86E4NB1M93UD|          5|            0|          0|   Y|
|R3DG8ASP4P2BYH|          3|            0|          0|   Y|
|R23DJKDOCX97I0|          1|            0|          0|   Y|
|R2PJOCRWZX4RQZ|          2|            1|          2|   Y|
|R2RRO66EPNH0SB|          5|            3|         15|   Y|
+--------------+-----------+-------------+-----------+----+
only showing top 10 rows



## Write DataFrame to RDS

In [0]:
#replace <your ...> with your database connection information
mode="append"
jdbc_url = "jdbc:postgresql://<your connection string>:5432/<your database-name>"
config = {"user":"<your database user>",
          "password": "<your database password>",
          "driver":"org.postgresql.Driver"}

In [0]:
vine_reviews_yes.write.jdbc(url=jdbc_url, table='video_game_vine_reviews', mode=mode, properties=config)